In [1]:
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input,Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update
import wget

In [2]:
spacex_df=wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df=pd.read_csv(spacex_df)
spacex_df.head()

100% [............................................................] 2476 / 2476

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [ ]:
#original
min_payload=spacex_df['Payload Mass (kg)'].min()
max_payload=spacex_df['Payload Mass (kg)'].max()


app = dash.Dash(__name__)


app.config.suppress_callback_exceptions = True


app.layout = html.Div(children=[ 
     html.H1('SpaceX Launch Performance ', 
        style={'textAlign': 'center',
               'color': '#503D36',
               'font-size': 24}),
    
    html.H2('Launch Site:', style={'margin-right': '2em'}),
    
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'},
                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                 ],
                 value='ALL',
                 placeholder="Select a Launch Site here",
                 searchable=True
                ),
    
    html.Br(),
    
    html.Div(dcc.Graph(id='success-pie-chart', 
                       style={'display': 'flex'})),
    
    html.Br(),
    
    
    
    html.H2('Choose Payload Mass (kg):',
            style={'margin-right': '2em'}),
    
    dcc.RangeSlider(id='payload-slider',
                        min=0, max=10000, step=1000,
                        marks={0: '0',
                               100: '100'},
                        value=[min_payload, max_payload]),
    
    html.Div(dcc.Graph(id='success-payload-scatter-chart',
                       style={'display': 'flex'}),
            ),
])

    

# Function decorator to specify function input and output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        #filtered_df=spacex_df.groupby('Launch Site',as_index=True).sum()['class']
        fig = px.pie(filtered_df, values='class', 
                     names='Launch Site', 
                     title='Total Success Launches By Site')
        return fig
    else:
        filtered_df=filtered_df[filtered_df['Launch Site']==entered_site]
        filtered_df=filtered_df.groupby(['class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count', 
                     names='class', 
                     title='Total Success Launches for site ' + entered_site)
        return fig
    
@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])
    

        # return the outcomes piechart for a selected site)

def get_scatter_chart(entered_site,payload_range):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload_range[0],payload_range[1])]
    if entered_site=='ALL':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Correlation between Payload Mass and Launch Success for all Sites')
        return fig
    else:
        filtered_df=filtered_df[filtered_df['Launch Site']==entered_site]
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success count on Payload mass for site ' + entered_site)
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
